# Imports

In [1]:
import pandas as pd
import requests
import datetime
# from concurrent.futures import ThreadPoolExecutor
from pprint import pprint
import os
os.environ['NUMEXPR_MAX_THREADS'] = str(os.cpu_count())

# Train schedule

In [2]:
## get train summary
url_sch = "https://www.irctc.co.in/eticketing/protected/mapps1/trnscheduleenquiry/"
headers_sch = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    # Requests sorts cookies= alphabetically
    # 'Cookie': '_ga=GA1.3.407357921.1653763540; __gpi=UID=000006f658163a2e:T=1656160588:RT=1656160588:S=ALNI_Ma-C--hdqqvJs0-5c5wrfXq4z-6ww; __gads=ID=e111572218788b3b-22873ecbffd200e1:T=1656160588:RT=1656160609:S=ALNI_MaLPcv2rgJnEFqdmc1DUwsljeLyGw; et_appVIP1=6ad0a3de765165ef2f70a399c5b50eb7ccecdf8f044b7d12f575d863b686396f06518232; _gid=GA1.3.907970754.1660014264; JSESSIONID=T8yAl1S_Iq4FP8jHFWGQUyKILFoIn52SKApQR3i3coFMbD21JrcE!-1623012044',
    'Referer': 'https://www.irctc.co.in/online-charts/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36 Edg/103.0.1264.77',
    'greq': '1660015243394',
    'sec-ch-ua': '" Not;A Brand";v="99", "Microsoft Edge";v="103", "Chromium";v="103"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}
train_no = 12214 ## DEE YPR Duronto
output_sch = requests.get(url_sch+str(train_no),headers=headers_sch)
pprint(output_sch.json())

{'duration': '0',
 'serverId': 'DM03AP14MS2',
 'stationFrom': 'DEE',
 'stationList': [{'arrivalTime': '--',
                  'dayCount': '1',
                  'departureTime': '22:10',
                  'distance': '0',
                  'haltTime': '--',
                  'routeNumber': '1',
                  'stationCode': 'DEE',
                  'stationName': 'DELHI S ROHILLA',
                  'stnSerialNumber': '1'},
                 {'arrivalTime': '03:50',
                  'dayCount': '2',
                  'departureTime': '03:55',
                  'distance': '423',
                  'haltTime': '05:00',
                  'routeNumber': '1',
                  'stationCode': 'VGLB',
                  'stationName': 'V LAKSHMIBAI',
                  'stnSerialNumber': '6'},
                 {'arrivalTime': '07:55',
                  'dayCount': '2',
                  'departureTime': '08:00',
                  'distance': '720',
                  'haltTime': '05:00',
    

In [3]:
rundays = {day:output_sch.json()[f'trainRunsOn{day}'] for day in ['Sun','Mon','Tue','Wed','Thu','Fri','Sat']}
rundays

{'Sun': 'N',
 'Mon': 'Y',
 'Tue': 'N',
 'Wed': 'N',
 'Thu': 'N',
 'Fri': 'N',
 'Sat': 'N'}

In [4]:
stations = pd.DataFrame(output_sch.json()['stationList'])
stations.dayCount = stations.dayCount.astype(int)
stations

,arrivalTime,dayCount,departureTime,distance,haltTime,routeNumber,stationCode,stationName,stnSerialNumber
0,--,1,22:10,0,--,1,DEE,DELHI S ROHILLA,1
1,03:50,2,03:55,423,05:00,1,VGLB,V LAKSHMIBAI,6
2,07:55,2,08:00,720,05:00,1,RKMP,RANI KAMALAPATI,8
3,16:40,2,16:45,1314,05:00,1,BPQ,BALHARSHAH,11
4,20:55,2,21:10,1680,15:00,1,SC,SECUNDERABAD JN,13
5,02:58,3,03:00,2091,02:00,1,GTL,GUNTAKAL JN,15
6,08:10,3,--,2367,--,1,YPR,YESVANTPUR JN,18


In [5]:
stations[stations['stationCode']=='VGLB'].dayCount.values[0]

2

In [6]:
station_map = stations.set_index('stationCode').stationName.to_dict()
station_map

{'DEE': 'DELHI S ROHILLA',
 'VGLB': 'V LAKSHMIBAI',
 'RKMP': 'RANI KAMALAPATI',
 'BPQ': 'BALHARSHAH',
 'SC': 'SECUNDERABAD JN',
 'GTL': 'GUNTAKAL JN',
 'YPR': 'YESVANTPUR JN'}

# Train composition

In [56]:
start_date = '2022-08-08' ## yyyy-mm-dd
st_dt = pd.to_datetime(start_date)
boarding_station = 'DEE' ## output_sch.json()['stationFrom']

In [57]:
if rundays[st_dt.day_name()[:3]] != 'Y':
    raise ValueError(f"Incorrect start date of train. This train runs on {[day for day in rundays if rundays[day]=='Y']}")

In [58]:
url_comp = 'https://www.irctc.co.in/online-charts/api/trainComposition'

headers_comp = {
    'Accept': 'application/json',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    # Already added when you pass json=
    # 'Content-Type': 'application/json',
    # Requests sorts cookies= alphabetically
    # 'Cookie': '_ga=GA1.3.407357921.1653763540; __gpi=UID=000006f658163a2e:T=1656160588:RT=1656160588:S=ALNI_Ma-C--hdqqvJs0-5c5wrfXq4z-6ww; __gads=ID=e111572218788b3b-22873ecbffd200e1:T=1656160588:RT=1656160609:S=ALNI_MaLPcv2rgJnEFqdmc1DUwsljeLyGw; et_appVIP1=6ad0a3de765165ef2f70a399c5b50eb7ccecdf8f044b7d12f575d863b686396f06518232; _gid=GA1.3.907970754.1660014264; JSESSIONID=gYOAwEFyJhR-M02NFXRLf98njfUy9sK9bAo2xplxQANj5QWj7ztL!-1623012044',
    'Origin': 'https://www.irctc.co.in',
    'Referer': 'https://www.irctc.co.in/online-charts/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36 Edg/103.0.1264.77',
    'sec-ch-ua': '" Not;A Brand";v="99", "Microsoft Edge";v="103", "Chromium";v="103"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

dayoffset = int(stations[stations['stationCode']==boarding_station].dayCount.values[0] - 1)
boarding_dt = str((pd.to_datetime(start_date) + datetime.timedelta(days=dayoffset)).date())

json_data_comp = {
    'trainNo': str(train_no),
    'jDate': boarding_dt,
    'boardingStation': boarding_station,
}


response_comp = requests.post(url_comp, headers=headers_comp, json=json_data_comp)
pprint(response_comp.json())

{'avlRemoteForBooking': 'BPQ',
 'cdd': [{'classCode': '2A',
          'coachName': 'A1',
          'positionFromEngine': 20,
          'vacantBerths': 3},
         {'classCode': '3A',
          'coachName': 'B10',
          'positionFromEngine': 8,
          'vacantBerths': 4},
         {'classCode': '3A',
          'coachName': 'B11',
          'positionFromEngine': 7,
          'vacantBerths': 22},
         {'classCode': '3A',
          'coachName': 'B12',
          'positionFromEngine': 6,
          'vacantBerths': 46},
         {'classCode': '3A',
          'coachName': 'B13',
          'positionFromEngine': 5,
          'vacantBerths': 45},
         {'classCode': '3A',
          'coachName': 'B14',
          'positionFromEngine': 4,
          'vacantBerths': 46},
         {'classCode': '3A',
          'coachName': 'B15',
          'positionFromEngine': 3,
          'vacantBerths': 46},
         {'classCode': '3A',
          'coachName': 'B16',
          'positionFromEngine': 2,
  

In [59]:
classes = set([item['classCode'] for item in response_comp.json()['cdd']])
classes

{'1A', '2A', '3A'}

# Vacant birth

In [60]:
url_vac = 'https://www.irctc.co.in/online-charts/api/vacantBerth'

headers_vac = {
    'Accept': 'application/json',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    # Already added when you pass json=
    # 'Content-Type': 'application/json',
    # Requests sorts cookies= alphabetically
    # 'Cookie': '_ga=GA1.3.407357921.1653763540; __gpi=UID=000006f658163a2e:T=1656160588:RT=1656160588:S=ALNI_Ma-C--hdqqvJs0-5c5wrfXq4z-6ww; __gads=ID=e111572218788b3b-22873ecbffd200e1:T=1656160588:RT=1656160609:S=ALNI_MaLPcv2rgJnEFqdmc1DUwsljeLyGw; et_appVIP1=6ad0a3de765165ef2f70a399c5b50eb7ccecdf8f044b7d12f575d863b686396f06518232; _gid=GA1.3.907970754.1660014264; JSESSIONID=5mqAxdueeXttqf1L7EnHwBpqwTAC2045HIwXqkUT0tHO82oTA-LH!-1623012044',
    'Origin': 'https://www.irctc.co.in',
    'Referer': 'https://www.irctc.co.in/online-charts/traincomposition',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36 Edg/103.0.1264.77',
    'sec-ch-ua': '" Not;A Brand";v="99", "Microsoft Edge";v="103", "Chromium";v="103"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

class_ = '3A'
boarding_station = 'DEE'

json_data_vac = {
    'trainNo': str(train_no),
    'boardingStation': boarding_station,
    'remoteStation': boarding_station,
    'trainSourceStation': output_sch.json()['stationFrom'],
    'jDate': start_date,
    'cls': class_,
    'chartType': 2,
}

response_vac = requests.post(url_vac, headers=headers_vac, json=json_data_vac)
pprint(response_vac.json().keys())
print('------------')
pprint(response_vac.json())

dict_keys(['vbd', 'error'])
------------
{'error': None,
 'vbd': [{'berthCode': 'U',
          'berthNumber': 19,
          'cabinCoupe': None,
          'cabinCoupeNo': None,
          'coachName': 'B1',
          'from': 'DEE',
          'splitNo': 1,
          'to': 'VGLB'},
         {'berthCode': 'L',
          'berthNumber': 20,
          'cabinCoupe': None,
          'cabinCoupeNo': None,
          'coachName': 'B1',
          'from': 'DEE',
          'splitNo': 1,
          'to': 'VGLB'},
         {'berthCode': 'L',
          'berthNumber': 1,
          'cabinCoupe': None,
          'cabinCoupeNo': None,
          'coachName': 'B1',
          'from': 'DEE',
          'splitNo': 1,
          'to': 'VGLB'},
         {'berthCode': 'M',
          'berthNumber': 2,
          'cabinCoupe': None,
          'cabinCoupeNo': None,
          'coachName': 'B1',
          'from': 'DEE',
          'splitNo': 1,
          'to': 'VGLB'},
         {'berthCode': 'U',
          'berthNumber': 3,
 

# Pandas dataframe of vacant seats

In [61]:
df = pd.DataFrame(response_vac.json()['vbd'])

In [62]:
df

,coachName,cabinCoupe,cabinCoupeNo,berthCode,berthNumber,from,to,splitNo
0,B1,None,None,U,19,DEE,VGLB,1
1,B1,None,None,L,20,DEE,VGLB,1
2,B1,None,None,L,1,DEE,VGLB,1
3,B1,None,None,M,2,DEE,VGLB,1
4,B1,None,None,U,3,DEE,VGLB,1
...,...,...,...,...,...,...,...,...
542,B8,None,None,U,46,DEE,VGLB,1
543,B8,None,None,P,8,DEE,VGLB,1
544,B8,None,None,R,31,DEE,VGLB,1
545,B8,None,None,U,43,DEE,VGLB,1


# Vacant count by birth

In [63]:
## U -> upper, M -> middle, L -> lower, SU -> P, SL -> R
berthmap = {'U':'UPPER','M':'MIDDLE','L':'LOWER','P':'SIDE UPPER','R':'SIDE LOWER'}

df.berthCode.value_counts()

U    188
M    178
P     76
L     57
R     48
Name: berthCode, dtype: int64

In [64]:
## vacant seats
sum(df.berthCode.value_counts())

547

In [65]:
pd.Series(index=list(map(lambda x: berthmap[x],df.berthCode.value_counts().index)),data = df.berthCode.value_counts().values, name='berth count')

UPPER         188
MIDDLE        178
SIDE UPPER     76
LOWER          57
SIDE LOWER     48
Name: berth count, dtype: int64

# Query

In [66]:
df[(df.coachName == 'B8') & (df.berthNumber==31)]

,coachName,cabinCoupe,cabinCoupeNo,berthCode,berthNumber,from,to,splitNo
544,B8,None,None,R,31,DEE,VGLB,1


In [67]:
df[(df.coachName == 'B8') & (df.berthCode.isin(['L','R']))]

,coachName,cabinCoupe,cabinCoupeNo,berthCode,berthNumber,from,to,splitNo
517,B8,None,None,R,39,DEE,VGLB,1
541,B8,None,None,R,23,DEE,VGLB,1
544,B8,None,None,R,31,DEE,VGLB,1


# Create aggregate dataset for all stations

In [81]:
# %%time
dfs = pd.DataFrame()
for station in stations.stationCode.unique():
    for class_type in classes:
        print(station,' ',class_type)
        
        json_data_vac = {
            'trainNo': str(train_no),
            'boardingStation': station,
            'remoteStation': station,
            'trainSourceStation': output_sch.json()['stationFrom'],
            'jDate': start_date,
            'cls': class_type,
            'chartType': 2,
        }
        
        response_vac = requests.post(url_vac, headers=headers_vac, json=json_data_vac)
        
        df_ = pd.DataFrame(response_vac.json()['vbd'])
        if len(df_):
            df_['berthName'] = df_['berthCode'].map(berthmap)
            df_['fromName'] = df_['from'].map(station_map)
            df_['toName'] = df_['to'].map(station_map)
            df_['class'] = class_type
            dfs = dfs.append(df_,ignore_index=True)
        else:
            print('Empty results')        

DEE   1A
DEE   3A
DEE   2A
VGLB   1A
VGLB   3A
VGLB   2A
Empty results
RKMP   1A
Empty results
RKMP   3A
RKMP   2A
BPQ   1A
Empty results
BPQ   3A
Empty results
BPQ   2A
Empty results
SC   1A
Empty results
SC   3A
Empty results
SC   2A
Empty results
GTL   1A
Empty results
GTL   3A
Empty results
GTL   2A
Empty results
YPR   1A
Empty results
YPR   3A
Empty results
YPR   2A
Empty results


In [82]:
dfs

,coachName,cabinCoupe,cabinCoupeNo,berthCode,berthNumber,from,to,splitNo,berthName,fromName,toName,class
0,H1,CB,C,U,10,DEE,VGLB,1,UPPER,DELHI S ROHILLA,V LAKSHMIBAI,1A
1,H1,CB,H,U,24,DEE,VGLB,1,UPPER,DELHI S ROHILLA,V LAKSHMIBAI,1A
2,H1,CB,H,L,23,DEE,VGLB,1,LOWER,DELHI S ROHILLA,V LAKSHMIBAI,1A
3,H1,CB,H,U,22,DEE,VGLB,1,UPPER,DELHI S ROHILLA,V LAKSHMIBAI,1A
4,H1,CP,G,U,20,DEE,VGLB,1,UPPER,DELHI S ROHILLA,V LAKSHMIBAI,1A
...,...,...,...,...,...,...,...,...,...,...,...,...
574,B10,None,None,R,71,RKMP,BPQ,1,SIDE LOWER,RANI KAMALAPATI,BALHARSHAH,3A
575,B10,None,None,L,52,RKMP,BPQ,1,LOWER,RANI KAMALAPATI,BALHARSHAH,3A
576,A1,None,None,U,46,RKMP,BPQ,1,UPPER,RANI KAMALAPATI,BALHARSHAH,2A
577,A1,None,None,U,44,RKMP,BPQ,1,UPPER,RANI KAMALAPATI,BALHARSHAH,2A


In [83]:
dfs.shape

(579, 12)

# Query

In [85]:
dfs.to.unique()

array(['VGLB', 'RKMP', 'BPQ'], dtype=object)

In [86]:
dfs['from'].unique()

array(['DEE', 'VGLB', 'RKMP'], dtype=object)

In [88]:
## Vacant lower
dfs[(dfs.coachName=='B8') & (dfs.berthName=='SIDE LOWER')]

,coachName,cabinCoupe,cabinCoupeNo,berthCode,berthNumber,from,to,splitNo,berthName,fromName,toName,class
523,B8,None,None,R,39,DEE,VGLB,1,SIDE LOWER,DELHI S ROHILLA,V LAKSHMIBAI,3A
547,B8,None,None,R,23,DEE,VGLB,1,SIDE LOWER,DELHI S ROHILLA,V LAKSHMIBAI,3A
550,B8,None,None,R,31,DEE,VGLB,1,SIDE LOWER,DELHI S ROHILLA,V LAKSHMIBAI,3A


In [90]:
dfs[dfs['class']=='1A']

,coachName,cabinCoupe,cabinCoupeNo,berthCode,berthNumber,from,to,splitNo,berthName,fromName,toName,class
0,H1,CB,C,U,10,DEE,VGLB,1,UPPER,DELHI S ROHILLA,V LAKSHMIBAI,1A
1,H1,CB,H,U,24,DEE,VGLB,1,UPPER,DELHI S ROHILLA,V LAKSHMIBAI,1A
2,H1,CB,H,L,23,DEE,VGLB,1,LOWER,DELHI S ROHILLA,V LAKSHMIBAI,1A
3,H1,CB,H,U,22,DEE,VGLB,1,UPPER,DELHI S ROHILLA,V LAKSHMIBAI,1A
4,H1,CP,G,U,20,DEE,VGLB,1,UPPER,DELHI S ROHILLA,V LAKSHMIBAI,1A
5,H1,CP,G,L,19,DEE,VGLB,1,LOWER,DELHI S ROHILLA,V LAKSHMIBAI,1A
564,H1,CP,G,U,20,VGLB,RKMP,1,UPPER,V LAKSHMIBAI,RANI KAMALAPATI,1A
565,H1,CP,G,L,19,VGLB,RKMP,1,LOWER,V LAKSHMIBAI,RANI KAMALAPATI,1A
